In [1]:
import pandas as pd
from glob import glob
import os

In [2]:
csv_files = glob('./data/census_tracts/*.csv')

csv_dict = {}
states_list = []

for filename in csv_files:
    filename_cleaned = os.path.basename(filename).replace('.csv', "")
    states_list.append(filename_cleaned.rpartition('_t')[0])
    filename_df = pd.read_csv(filename)
    csv_dict[filename_cleaned] = filename_df
    


In [3]:
len(csv_dict)

43

In [4]:
states_list.pop(0)

'MN'

In [5]:
tracts_df = csv_dict['MN_tracts']

for state in states_list:
    tracts_df = pd.concat([tracts_df, csv_dict[state+'_tracts']])

In [6]:
tracts_df['name'] = tracts_df['name'].astype(str)+'_'+tracts_df['parent.location'].str.rpartition(', ')[0]



In [7]:
tracts_df['state'] = tracts_df['parent.location'].str.rpartition(', ')[2]

In [8]:
tracts_df

,GEOID,year,name,parent.location,population,poverty.rate,renter.occupied.households,pct.renter.occupied,median.gross.rent,median.household.income,...,pct.multiple,pct.other,eviction.filings,evictions,eviction.rate,eviction.filing.rate,low.flag,imputed,subbed,state
0,27003050107,2012,501.07_Anoka County,"Anoka County, Minnesota",2832.0,0.81,83.0,9.99,1047.0,76406.0,...,1.80,0.25,11.0,8.0,9.64,13.25,0,0,0,Minnesota
1,27003050108,2012,501.08_Anoka County,"Anoka County, Minnesota",4931.0,8.39,311.0,19.55,909.0,65313.0,...,3.69,0.18,19.0,11.0,3.54,6.11,0,0,0,Minnesota
2,27003050109,2012,501.09_Anoka County,"Anoka County, Minnesota",5141.0,2.45,87.0,3.43,1640.0,94821.0,...,1.34,0.00,8.0,3.0,3.45,9.20,0,0,0,Minnesota
3,27003050110,2012,501.1_Anoka County,"Anoka County, Minnesota",2665.0,8.26,53.0,4.44,1327.0,81761.0,...,0.53,0.00,5.0,3.0,5.63,9.38,0,0,0,Minnesota
4,27003050111,2012,501.11_Anoka County,"Anoka County, Minnesota",3819.0,1.11,74.0,9.83,1247.0,82847.0,...,1.62,0.00,13.0,5.0,6.73,17.49,0,0,0,Minnesota
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5191,36085990100,2003,9901.0_Richmond County,"Richmond County, New York",0.0,0.00,0.0,0.00,0.0,0.0,...,0.00,0.00,0.0,0.0,0.00,0.00,0,0,0,New York
5192,36085990100,2004,9901.0_Richmond County,"Richmond County, New York",0.0,0.00,0.0,0.00,0.0,0.0,...,0.00,0.00,0.0,0.0,0.00,0.00,0,0,0,New York
5193,36085990100,2005,9901.0_Richmond County,"Richmond County, New York",0.0,0.00,0.0,0.00,0.0,0.0,...,0.00,0.00,0.0,0.0,0.00,0.00,0,0,0,New York
5194,36085990100,2006,9901.0_Richmond County,"Richmond County, New York",0.0,0.00,0.0,0.00,0.0,0.0,...,0.00,0.00,0.0,0.0,0.00,0.00,0,0,0,New York


In [9]:
tracts_df.to_csv('tracts_annual')

In [10]:
tract_groupby = tracts_df.groupby(['name', 'state'], as_index=False).median()

In [11]:
null_groupby = tract_groupby[tract_groupby['eviction.rate'].isna()]

for index, row in null_groupby.iterrows():
        tract_groupby.drop(index, inplace=True)
        
tract_groupby.drop(['low.flag', 'imputed', 'subbed'], axis=1, inplace=True)

tract_groupby.drop(tract_groupby[tract_groupby['population'] == 0].index, inplace=True)

tract_groupby.dropna(inplace=True)

In [12]:
tract_groupby

,name,state,GEOID,year,population,poverty.rate,renter.occupied.households,pct.renter.occupied,median.gross.rent,median.household.income,...,pct.hispanic,pct.am.ind,pct.asian,pct.nh.pi,pct.multiple,pct.other,eviction.filings,evictions,eviction.rate,eviction.filing.rate
0,1.01_Ashtabula County,Ohio,3.900700e+10,2012.0,4824.00,15.92,482.0,31.30,377.0,32277.0,...,5.83,0.27,0.27,0.00,1.47,0.00,20.0,7.0,1.82,4.270
1,1.01_Brazos County,Texas,4.804100e+10,2005.0,6122.16,11.20,728.0,24.73,880.0,58634.0,...,17.35,0.00,0.04,0.00,0.79,0.00,17.0,6.0,0.82,2.340
2,1.01_Cache County,Utah,4.900500e+10,2014.5,4872.00,6.68,285.0,15.23,673.0,52075.0,...,5.85,0.12,0.08,0.02,0.49,0.00,4.0,3.0,1.05,1.395
3,1.01_Clark County,Nevada,3.200300e+10,2013.0,6757.00,11.76,485.0,26.17,1189.0,46843.0,...,63.40,0.38,3.18,2.47,1.26,0.00,43.0,24.0,4.84,8.680
4,1.01_Durham County,North Carolina,3.706300e+10,2011.5,2895.00,28.02,481.5,47.89,694.0,35467.0,...,19.93,0.00,0.00,0.00,3.73,0.00,159.0,34.0,6.89,33.250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37607,990.0_Kings County,New York,3.604710e+10,2005.0,1473.00,10.88,209.0,41.76,1020.0,59306.0,...,6.38,0.00,4.68,0.00,0.00,0.00,15.0,8.0,3.80,7.130
37741,992.0_Kings County,New York,3.604710e+10,2005.0,1467.00,4.09,253.0,43.48,1193.0,69965.0,...,2.25,0.00,1.23,0.00,0.00,0.00,10.0,6.0,2.36,3.970
37743,994.0_Kings County,New York,3.604710e+10,2005.0,2195.00,3.85,295.0,49.31,1413.0,82903.0,...,11.44,0.00,0.00,0.00,0.00,0.00,31.0,9.0,3.07,10.590
37744,996.0_Kings County,New York,3.604710e+10,2005.0,3986.00,3.06,565.0,46.02,1266.0,67757.0,...,3.46,0.00,0.00,0.00,0.00,0.45,27.0,9.0,1.59,4.770


In [18]:
tract_mean = tracts_df.groupby(['name', 'state'], as_index=False).mean()

for index, row in null_groupby.iterrows():
        tract_mean.drop(index, inplace=True)
        
tract_mean.drop(['low.flag', 'imputed', 'subbed'], axis=1, inplace=True)

tract_mean.drop(tract_groupby[tract_groupby['population'] == 0].index, inplace=True)

tract_mean.dropna(inplace=True)

In [19]:
tract_mean

,name,state,GEOID,year,population,poverty.rate,renter.occupied.households,pct.renter.occupied,median.gross.rent,median.household.income,...,pct.hispanic,pct.am.ind,pct.asian,pct.nh.pi,pct.multiple,pct.other,eviction.filings,evictions,eviction.rate,eviction.filing.rate
0,1.01_Ashtabula County,Ohio,3.900700e+10,2009.666667,4237.666667,15.416667,457.777778,30.570000,369.000000,31828.333333,...,4.103333,0.256667,0.256667,0.00,1.403333,0.000000,17.666667,7.666667,1.628889,3.858889
1,1.01_Brazos County,Texas,4.804100e+10,2005.333333,5763.093333,10.490000,731.666667,24.216667,802.666667,55900.000000,...,14.703333,0.076667,0.180000,0.00,0.826667,0.026667,21.000000,8.000000,1.080000,2.853333
2,1.01_Cache County,Utah,4.900500e+10,2014.500000,4872.000000,6.680000,285.000000,15.230000,673.000000,52075.000000,...,5.850000,0.120000,0.080000,0.02,0.490000,0.000000,4.000000,3.000000,1.050000,1.395000
3,1.01_Clark County,Nevada,3.200300e+10,2013.000000,6757.000000,11.760000,484.666667,26.170000,1189.000000,46843.000000,...,63.400000,0.380000,3.180000,2.47,1.260000,0.000000,39.333333,23.666667,4.876667,8.103333
4,1.01_Durham County,North Carolina,3.706300e+10,2011.100000,2907.503000,23.682000,492.200000,43.092000,765.200000,37424.397000,...,20.623000,0.006000,0.117000,0.00,3.686000,0.029000,173.000000,37.900000,7.874000,35.982000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37740,9917.03_Dane County,Wisconsin,5.502599e+10,2001.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
37741,992.0_Kings County,New York,3.604710e+10,2005.000000,1593.666667,6.013333,253.000000,42.733333,1092.333333,62094.666667,...,5.146667,0.000000,2.823333,0.00,1.353333,0.126667,11.000000,5.333333,2.106667,4.346667
37743,994.0_Kings County,New York,3.604710e+10,2005.000000,2106.666667,6.590000,295.000000,49.140000,1246.666667,71813.666667,...,10.613333,0.120000,0.690000,0.00,1.553333,0.053333,32.333333,13.666667,4.620000,10.950000
37744,996.0_Kings County,New York,3.604710e+10,2005.000000,3949.000000,6.526667,564.666667,46.363333,1141.000000,60286.333333,...,4.250000,0.016667,0.680000,0.00,0.463333,0.576667,25.333333,9.333333,1.650000,4.486667


In [37]:
tract_mean[tract_mean['eviction.rate'] < 69]['eviction.rate'].max()

60.7

In [33]:
tract_groupby[tract_groupby['eviction.rate'] < 90]['eviction.rate'].max()

57.5

In [39]:
tract_mean[tract_mean['eviction.rate'] == 60.7]

,name,state,GEOID,year,population,poverty.rate,renter.occupied.households,pct.renter.occupied,median.gross.rent,median.household.income,...,pct.hispanic,pct.am.ind,pct.asian,pct.nh.pi,pct.multiple,pct.other,eviction.filings,evictions,eviction.rate,eviction.filing.rate
22072,435.0_Bronx County,New York,3.600504e+10,2006.75,41.415,50.0,5.0,63.2325,186.0,34493.5,...,52.485,0.0,0.0,0.0,2.535,2.175,12.25,3.0,60.7,246.02


In [40]:
tract_groupby[tract_groupby['name'] == '435.0_Bronx County']

,name,state,GEOID,year,population,poverty.rate,renter.occupied.households,pct.renter.occupied,median.gross.rent,median.household.income,...,pct.hispanic,pct.am.ind,pct.asian,pct.nh.pi,pct.multiple,pct.other,eviction.filings,evictions,eviction.rate,eviction.filing.rate
22072,435.0_Bronx County,New York,3.600504e+10,2005.5,38.59,50.0,5.0,52.78,0.0,25059.0,...,66.615,0.0,0.0,0.0,0.0,0.0,3.0,2.0,41.4,61.795


In [ ]:
for index, row in null_groupby.iterrows():
        tract_groupby.drop(index, inplace=True)

In [ ]:
tract_groupby.drop(['low.flag', 'imputed', 'subbed'], axis=1, inplace=True)

In [ ]:
tract_groupby

In [ ]:
tract_groupby[tract_groupby['population'] == 0]

In [ ]:
tract_groupby[tract_groupby['population'] == 0].describe()

In [ ]:
tract_groupby.drop(tract_groupby[tract_groupby['population'] == 0].index, inplace=True)

In [ ]:
tract_groupby

In [ ]:
tract_groupby.info()

In [ ]:
tract_groupby.dropna(inplace=True)

In [ ]:
tract_groupby.to_csv('cleaned_tracts.csv')

In [ ]:
tract_groupby

In [ ]:
tract_groupby[tract_groupby['state'] == 'Wyoming']

In [ ]:
tract_groupby[tract_groupby['GEOID'] == 56043000200]

In [13]:
gender_df = pd.read_csv('./data/dataframes/all_gender')

In [14]:
gender_df.drop('Unnamed: 0', axis=1, inplace=True)

In [15]:
gender_df = gender_df.groupby(by='GEO_ID', as_index=False).median()

In [16]:
gender_df.rename(columns={'GEO_ID': 'GEOID'}, inplace=True)

In [ ]:
tract_groupby

In [17]:
merged = pd.merge(tract_groupby, gender_df, on='GEOID')

In [18]:
merged.drop(['year_x', 'year_y'], axis=1, inplace=True)

In [19]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36058 entries, 0 to 36057
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   name                        36058 non-null  object 
 1   state                       36058 non-null  object 
 2   GEOID                       36058 non-null  float64
 3   population                  36058 non-null  float64
 4   poverty.rate                36058 non-null  float64
 5   renter.occupied.households  36058 non-null  float64
 6   pct.renter.occupied         36058 non-null  float64
 7   median.gross.rent           36058 non-null  float64
 8   median.household.income     36058 non-null  float64
 9   median.property.value       36058 non-null  float64
 10  rent.burden                 36058 non-null  float64
 11  pct.white                   36058 non-null  float64
 12  pct.af.am                   36058 non-null  float64
 13  pct.hispanic                360

In [ ]:
merged.to_csv('./data/dataframes/final_tracts.csv')

In [ ]:
tract_mean = tracts_df.groupby(['name', 'state'], as_index=False).mean()

In [ ]:
null_groupby = tract_mean[tract_mean['eviction.rate'].isna()]

In [ ]:
for index, row in null_groupby.iterrows():
        tract_mean.drop(index, inplace=True)

In [ ]:
tract_mean.drop(['low.flag', 'imputed', 'subbed'], axis=1, inplace=True)

In [ ]:
tract_mean.drop(tract_mean[tract_mean['population'] == 0].index, inplace=True)

In [ ]:
tract_mean.dropna(inplace=True)

In [ ]:
tract_mean.describe()

In [ ]:
tract_groupby.describe()

In [ ]:
tract_mean[tract_mean['eviction.rate'] > 100]

In [ ]:
tract_groupby[tract_groupby['eviction.rate'] < 100]['eviction.rate'].max()

In [ ]:
tract_mean[tract_mean['eviction.rate'] < 100]['eviction.rate'].max()

In [ ]:
tract_mean[tract_mean['eviction.rate'] == 95.80555555555556]

In [ ]:
tract_mean

In [ ]:
tract_groupby